In [1]:
import os

In [2]:
%pwd

'c:\\Users\\abdus samad\\Desktop\\Churn_ML\\Churn_ML_Project\\research'

In [8]:
os.chdir("Churn_ML_Project")
%pwd

'c:\\Users\\abdus samad\\Desktop\\Churn_ML\\Churn_ML_Project'

In [45]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    dataset_slug: str
    local_data_file: Path
    unzip_dir: Path

In [40]:
from src.Churn_Predictor.constants import *
from src.Churn_Predictor.utils.common import read_yaml, create_directories
from pathlib import Path


class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            dataset_slug=config.dataset_slug,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [41]:
import os
import urllib.request as request
from src.Churn_Predictor import logger
import zipfile

In [47]:
import os
import zipfile
from src.Churn_Predictor import logger


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self):
        """Download datasets from kaggle using kaggle API"""
        if not os.path.exists(self.config.local_data_file):
            try:
                from kaggle.api.kaggle_api_extended import KaggleApi

                api = KaggleApi()
                api.authenticate()

                logger.info(f"Downloading dataset: {self.config.dataset_slug}")
                api.dataset_download_files(
                    self.config.dataset_slug, 
                    path=self.config.root_dir, 
                    unzip=False
                )
                
                # Rename the downloaded file
                downloaded_file = os.path.join(
                    self.config.root_dir, 
                    f"{self.config.dataset_slug.split('/')[-1]}.zip"
                )
                
                if os.path.exists(downloaded_file):
                    os.rename(downloaded_file, self.config.local_data_file)
                
                logger.info(f"Dataset downloaded successfully to: {self.config.local_data_file}")

            except Exception as e:
                logger.error(f"Error downloading dataset: {e}")
                raise e
        else:
            logger.info(f"Dataset already exists at: {self.config.local_data_file}")
    
    def extract_zip_file(self):  # ← Fixed indentation (was nested inside download_data)
        """Extract the downloaded zip file"""
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger.info(f"Dataset extracted successfully to: {unzip_path}")
        except Exception as e:
            logger.error(f"Error extracting dataset: {e}")
            raise e
    
    def fetch_and_prepare_data(self):
        """Main method: download and extract data"""
        self.download_data()
        self.extract_zip_file()

In [49]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"Error in data ingestion: {e}")
    raise e

[2026-02-12 23:49:40,353: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-02-12 23:49:40,355: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-12 23:49:40,356: INFO: common: yaml file: schema.yaml loaded successfully]
[2026-02-12 23:49:40,357: INFO: common: created directory at: artifacts]
[2026-02-12 23:49:40,359: INFO: common: created directory at: artifacts/data_ingestion]
[2026-02-12 23:49:40,361: INFO: 2611973756: Downloading dataset: blastchar/telco-customer-churn]
Dataset URL: https://www.kaggle.com/datasets/blastchar/telco-customer-churn
[2026-02-12 23:49:43,248: INFO: 2611973756: Dataset downloaded successfully to: artifacts/data_ingestion/data.zip]
[2026-02-12 23:49:43,280: INFO: 2611973756: Dataset extracted successfully to: artifacts/data_ingestion/raw_data]
